Below, we demonstrate distribution calibration of probabilistic uncertainty over continuous output. 

At first, we import the necessary files. For this demo, we use the  [California Housing Dataset](https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html). 

In [1]:
import torch
import sys
sys.path.append('../..')

from torchuq.transform.distcal_continuous import *
from torchuq.transform.calibrate import *
from torchuq.evaluate.distribution_cal import *
from torchuq.dataset.regression import *
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_absolute_error
from torchuq.evaluate import quantile as q_eval
from matplotlib import pyplot as plt

#uci_dataset = ["wine", "crime", "naval", "protein", "superconductivity"]

subset_uci = ["cal_housing"]



Below, we use Bayesian Ridge Regression as the base model to predict probabilistic outcome, represented by the mean and standard deviation as parameters of a Gaussian outcome distribution.

We use object of the **DistCalibrator** class to train a recalibrator that takes the probabilistic outcome from the base model and outputs the recalibrated distribution parameterized by a fixed number of equispaced quantiles. In this example, we use 20 equispaced quantiles to featurize the outcome distribution. 

We use an independent calibration dataset to train the DistCalibrator. We evaluate the quality of probabilistic uncertainty with the check score and calibration score as defined [here](https://arxiv.org/pdf/2112.07184). 

In [2]:
# # Number of evaluation buckets
# num_buckets=20

# for name in subset_uci:
# 	# 60% Train, 20% Calibration, 20% Test dataset
# 	dataset = get_regression_datasets(name, val_fraction=0.2, test_fraction=0.2, split_seed=0, normalize=True, verbose=True)
	
# 	train_dataset, cal_dataset, test_dataset = dataset
# 	X_train, y_train = train_dataset[:][0], train_dataset[:][1]
# 	X_cal, y_cal = cal_dataset[:][0], cal_dataset[:][1]
# 	X_test, y_test = test_dataset[:][0], test_dataset[:][1]
	
# 	# Bayesian Ridge Regression to obtain probabilistic outcomes parameterized by the mean and std deviation of Gaussian outcome for each data-point
# 	reg = BayesianRidge().fit(X_train, y_train)
# 	print(f"Coeff of determination (R^2) on Train: {reg.score(X_train, y_train):.2}")
# 	print(f"Coeff of determination (R^2) on Test: {reg.score(X_test, y_test):.2}")
	


# 	# Predict mean and std deviation of the outcome distribution on the calibration and test datasets 
# 	mean_cal, std_dev_cal = reg.predict(X_cal.numpy(), return_std=True)
# 	mean_cal, std_dev_cal = torch.Tensor(mean_cal), torch.Tensor(std_dev_cal)

# 	mean_test, std_dev_test = reg.predict(X_test.numpy(), return_std=True)
# 	mean_test, std_dev_test = torch.Tensor(mean_test), torch.Tensor(std_dev_test)

# 	params_cal = torch.cat((mean_cal.reshape(-1, 1), std_dev_cal.reshape(-1, 1)), axis=1)
# 	params_test = torch.cat((mean_test.reshape(-1, 1), std_dev_test.reshape(-1, 1)), axis=1)

# 	# Convert probabilistic predictions to quantiles
# 	quantiles_cal = convert_normal_to_quantiles(mean_cal, std_dev_cal, num_buckets)
# 	quantiles_test = convert_normal_to_quantiles(mean_test, std_dev_test, num_buckets)

	

# 	# Use the DistCalibrator class and train it on the calibration dataset
# 	# Here, the recalibrator uses a fixed number of equispaced quantiles as featurization of the probabilistic outcome
# 	calibrator = DistCalibrator(num_buckets = num_buckets, quantile_input=True, verbose=True)
# 	calibrator.train(quantiles_cal, torch.Tensor(y_cal))

# 	quantile_calibrator = RegressionCalibrator()
# 	input_cdf = torch.distributions.Normal(mean_cal, std_dev_cal).cdf(y_cal)
# 	empirical_cdf = compute_empirical_cdf(input_cdf)
# 	quantile_calibrator.fit(input_cdf, empirical_cdf)


# 	calibrated_quantiles_cal = convert_normal_cdf_to_quantiles(mean_cal, std_dev_cal, quantile_calibrator.inverse_calibrator(num_buckets=num_buckets))
# 	calibrated_quantiles_test = convert_normal_cdf_to_quantiles(mean_test, std_dev_test, quantile_calibrator.inverse_calibrator(num_buckets=num_buckets))

# 	# Below code is needed if you featurized the Gaussian probabilistic outcome using their parameters mean and std deviation
# 	# calibrator = DistCalibrator(quantile_input=False, verbose=True)
# 	# calibrator.train(params_cal, torch.Tensor(y_cal))

# 	# Evaluation
# 	# 
	
# 	# Compare check scores and weighted calibrations cores 
# 	print("="*25)
# 	check_score_before, check_score_after = comparison_quantile_check_score(quantiles_cal, torch.Tensor(y_cal), np.linspace(0, 1, num_buckets), model=calibrator)

# 	_ , check_score_baseline = comparison_quantile_check_score(quantiles_cal, torch.Tensor(y_cal), np.linspace(0, 1, num_buckets), quant_calibrated_outcome=calibrated_quantiles_cal)

# 	print(f"[Calibration Split] Check score before calibration={check_score_before}, Check score after quantile calibration={check_score_baseline}, Check score after dist calibration={check_score_after}")


# 	cal_score_before, cal_score_after = comparison_quantile_calibration_scores(quantiles_cal, torch.Tensor(y_cal), np.linspace(0, 1, num_buckets), model=calibrator)
# 	cal_score_baseline, _ = comparison_quantile_calibration_scores(quantiles_cal, torch.Tensor(y_cal), np.linspace(0, 1, num_buckets), model=None, old_quantiles=calibrated_quantiles_cal)

# 	print(f"[Calibration Split] Calibration score before calibration={cal_score_before}, Calibration score after quantile calibration={cal_score_baseline} , Calibration score after dist calibration={cal_score_after}")

# 	print("="*25)

# 	check_score_before, check_score_after = comparison_quantile_check_score(quantiles_test, torch.Tensor(y_test), np.linspace(0, 1, num_buckets), model=calibrator)

# 	_, check_score_baseline = comparison_quantile_check_score(quantiles_test, torch.Tensor(y_test), np.linspace(0, 1, num_buckets), quant_calibrated_outcome=calibrated_quantiles_test)

# 	print(f"[Test Split] Check score before calibration={check_score_before}, Check score after quantile calibration={cal_score_baseline}, Check score after dist calibration={check_score_after}")

# 	cal_score_before, cal_score_after = comparison_quantile_calibration_scores(quantiles_test, torch.Tensor(y_test), np.linspace(0, 1, num_buckets), model=calibrator)

# 	cal_score_baseline, _ = comparison_quantile_calibration_scores(quantiles_test, torch.Tensor(y_test), np.linspace(0, 1, num_buckets), model=None, old_quantiles=calibrated_quantiles_test)
	
# 	print(f"[Test Split] Calibration score before calibration={cal_score_before}, Calibration score after quantile calibration={cal_score_baseline}, Calibration score after dist calibration={cal_score_after}")
# 	print("="*25)



In [3]:
# Number of evaluation buckets
num_buckets=20
name = "cal_housing"

# 60% Train, 20% Calibration, 20% Test dataset
dataset = get_regression_datasets(name, val_fraction=0.2, test_fraction=0.2, split_seed=0, normalize=True, verbose=True)

train_dataset, cal_dataset, test_dataset = dataset
X_train, y_train = train_dataset[:][0], train_dataset[:][1]
X_cal, y_cal = cal_dataset[:][0], cal_dataset[:][1]
X_test, y_test = test_dataset[:][0], test_dataset[:][1]

# Bayesian Ridge Regression to obtain probabilistic outcomes parameterized by the mean and std deviation of Gaussian outcome for each data-point
reg = BayesianRidge().fit(X_train, y_train)
print(f"Coeff of determination (R^2) on Train: {reg.score(X_train, y_train):.2}")
print(f"Coeff of determination (R^2) on Test: {reg.score(X_test, y_test):.2}")



# Predict mean and std deviation of the outcome distribution on the calibration and test datasets 
mean_cal, std_dev_cal = reg.predict(X_cal.numpy(), return_std=True)
mean_cal, std_dev_cal = torch.Tensor(mean_cal), torch.Tensor(std_dev_cal)

mean_test, std_dev_test = reg.predict(X_test.numpy(), return_std=True)
mean_test, std_dev_test = torch.Tensor(mean_test), torch.Tensor(std_dev_test)

params_cal = torch.cat((mean_cal.reshape(-1, 1), std_dev_cal.reshape(-1, 1)), axis=1)
params_test = torch.cat((mean_test.reshape(-1, 1), std_dev_test.reshape(-1, 1)), axis=1)

# Convert probabilistic predictions to quantiles
quantiles_cal = convert_normal_to_quantiles(mean_cal, std_dev_cal, num_buckets)
quantiles_test = convert_normal_to_quantiles(mean_test, std_dev_test, num_buckets)



# Use the DistCalibrator class and train it on the calibration dataset
# Here, the recalibrator uses a fixed number of equispaced quantiles as featurization of the probabilistic outcome
calibrator = DistCalibrator(num_buckets = num_buckets, quantile_input=True, verbose=True)
calibrator.train(quantiles_cal, torch.Tensor(y_cal), num_epochs=10)



Loading dataset cal_housing....
Splitting into train/val/test with 12384/4128/4128 samples
Done loading dataset cal_housing
Coeff of determination (R^2) on Train: 0.61
Coeff of determination (R^2) on Test: 0.6


100%|██████████| 1500/1500 [00:22<00:00, 66.35it/s]


In [4]:
# from torchuq.transform.distcal_continuous import convert_normal_cdf_to_quantiles
# from torchuq.transform.distcal_continuous import convert_normal_to_quantiles

In [32]:
def compute_empirical_cdf(cdf):
    """
    This function takes the output CDF as produced by probabilistic model corresponding to each observed outcome in the dataset to produce empirical CDF 
    This will be used to perform quantile calibration in Algorithm 1 of Kuleshov 2018
    """

    empirical_cdf = (torch.Tensor([torch.sum(cdf<=p) for p in cdf]))/len(cdf)

    return empirical_cdf
def convert_normal_cdf_to_quantiles(mean, std_dev, cdf_values):
    """ Converts prediction in the form of Normal distribution over outcomes to equispaced quantiles equal to num_buckets

        Args:
            mean (tensor): a batch of scalar means
            std_dev (tensor): a batch of scalar outcomes
            
        Returns:
            tensor: batch of cdf predictions represented as equispaced quantiles with the shape [batch_size, num_buckets]  
        """
    normal_dist = Normal(mean, std_dev)
    # print(cdf_values)
    # print(mean.shape, cdf_values.shape)
    quantiles=torch.zeros((mean.shape[0], cdf_values.shape[0]))

    for i, cdf_value in enumerate(cdf_values):

        quantiles[:, i] = normal_dist.icdf(cdf_value).T
    quantiles[:, -1] = mean+6*std_dev 
    return quantiles
def comparison_quantile_check_score(x_data, y_data, taus=np.linspace(0, 1, num=11), model=None, quant_calibrated_outcome=None):
    """ 
    Computes check scores before and after applying recalibrator in the continuous outcome setting where distribution is featurized using equispaced quantiles. 
    """
    if x_data is None and y_data is None:
        return 0, 0
    check_score_before, check_score_after = 0, 0
    for i, tau in enumerate(taus):
        temp_before = check_score(tau, x_data[:, i], y_data).mean()
        if model:
          calibrated_outcome = model.model.predict((tau, x_data))[0]
        else:
          calibrated_outcome = quant_calibrated_outcome[:, i]
          # calibrated_outcome = x_data[:, i]
        temp_after = check_score(tau, calibrated_outcome, y_data).mean()
        if not model and i>18:
            print(i, tau, calibrated_outcome.detach()[:3], y_data.detach()[:3], temp_after.detach())
        check_score_before += temp_before
        check_score_after += temp_after
    print(check_score_before, check_score_after.detach())
    return check_score_before, check_score_after.detach()

def comparison_quantile_calibration_scores(x_data, y_data, taus=np.linspace(0, 1, num=11), model=None, old_quantiles=None):
    """ 
    Computes weighted calibration scores before and after applying recalibrator in the continuous outcome setting where distribution is featurized using equispaced quantiles. 
    """
    # Todo: remove redundant parts
    if x_data is None and y_data is None:
        return (0, 0)
    
    if old_quantiles is None:
      old_quantiles = x_data
    
    num_buckets = len(taus)
    expected_tau = taus
    empirical_tau = np.zeros(num_buckets)
    empirical_tau_old = np.zeros(num_buckets)
    for tau_i, tau in enumerate(expected_tau):
        q_old = old_quantiles[:, tau_i]
        if model:
          q_new = model.model.predict((tau, x_data))[0]
        else:
          q_new = q_old
       

        # Todo: write efficient code
        for outcome_i, outcome in enumerate(y_data):

            if(outcome<=q_new[outcome_i]):
                empirical_tau[tau_i]+=1
            if(outcome<=q_old[outcome_i]):
                empirical_tau_old[tau_i]+=1
        empirical_tau[tau_i]/=len(y_data)
        empirical_tau_old[tau_i]/=len(y_data)

        empirical_tau[len(empirical_tau)-1]=1
        empirical_tau_old[len(empirical_tau)-1]=1
        cal_score_before = (((empirical_tau_old-expected_tau)**2)*empirical_tau_old).sum()
        cal_score_after = (((empirical_tau-expected_tau)**2)*empirical_tau).sum()

    return (cal_score_before, cal_score_after)

quantile_calibrator = RegressionCalibrator()
input_cdf = torch.distributions.Normal(mean_cal, std_dev_cal).cdf(y_cal)
empirical_cdf = compute_empirical_cdf(input_cdf)
quantile_calibrator.train(input_cdf, empirical_cdf)


calibrated_quantiles_cal = convert_normal_cdf_to_quantiles(mean_cal, std_dev_cal, torch.Tensor(quantile_calibrator.inverse_calibrator(num_buckets=num_buckets)))
calibrated_quantiles_test = convert_normal_cdf_to_quantiles(mean_test, std_dev_test, torch.Tensor(quantile_calibrator.inverse_calibrator(num_buckets=num_buckets)))

# Below code is needed if you featurized the Gaussian probabilistic outcome using their parameters mean and std deviation
# calibrator = DistCalibrator(quantile_input=False, verbose=True)
# calibrator.train(params_cal, torch.Tensor(y_cal))

# Evaluation
# 

# Compare check scores and weighted calibrations cores 
print("="*25)
check_score_before, check_score_after = comparison_quantile_check_score(quantiles_cal, torch.Tensor(y_cal), np.linspace(0, 1, num_buckets), model=calibrator)

_ , check_score_baseline = comparison_quantile_check_score(quantiles_cal, torch.Tensor(y_cal), np.linspace(0, 1, num_buckets), quant_calibrated_outcome=calibrated_quantiles_cal)

print(f"[Calibration Split] Check score before calibration={check_score_before}, Check score after quantile calibration={check_score_baseline}, Check score after dist calibration={check_score_after}")


cal_score_before, cal_score_after = comparison_quantile_calibration_scores(quantiles_cal, torch.Tensor(y_cal), np.linspace(0, 1, num_buckets), model=calibrator)
cal_score_baseline, _ = comparison_quantile_calibration_scores(quantiles_cal, torch.Tensor(y_cal), np.linspace(0, 1, num_buckets), model=None, old_quantiles=calibrated_quantiles_cal)

print(f"[Calibration Split] Calibration score before calibration={cal_score_before}, Calibration score after quantile calibration={cal_score_baseline} , Calibration score after dist calibration={cal_score_after}")

print("="*25)

check_score_before, check_score_after = comparison_quantile_check_score(quantiles_test, torch.Tensor(y_test), np.linspace(0, 1, num_buckets), model=calibrator)

_, check_score_baseline = comparison_quantile_check_score(quantiles_test, torch.Tensor(y_test), np.linspace(0, 1, num_buckets), quant_calibrated_outcome=calibrated_quantiles_test)

print(f"[Test Split] Check score before calibration={check_score_before}, Check score after quantile calibration={check_score_baseline}, Check score after dist calibration={check_score_after}")

cal_score_before, cal_score_after = comparison_quantile_calibration_scores(quantiles_test, torch.Tensor(y_test), np.linspace(0, 1, num_buckets), model=calibrator)

cal_score_baseline, _ = comparison_quantile_calibration_scores(quantiles_test, torch.Tensor(y_test), np.linspace(0, 1, num_buckets), model=None, old_quantiles=calibrated_quantiles_test)

print(f"[Test Split] Calibration score before calibration={cal_score_before}, Calibration score after quantile calibration={cal_score_baseline}, Calibration score after dist calibration={cal_score_after}")
print("="*25)



tensor(3.1940) tensor(2.9693)
19 1.0 tensor([3.9249, 4.3757, 3.6061]) tensor([-0.6072,  0.2980, -0.0540]) tensor(0.)
tensor(3.1940) tensor(3.1263)
[Calibration Split] Check score before calibration=3.1939690113067627, Check score after quantile calibration=3.1262903213500977, Check score after dist calibration=2.9693214893341064
[Calibration Split] Calibration score before calibration=0.04242164668846845, Calibration score after quantile calibration=1.8836053354855378e-07 , Calibration score after dist calibration=0.009871298769362314
tensor(3.1997) tensor(2.9658)
19 1.0 tensor([3.4195, 4.4616, 3.8689]) tensor([ 2.5410,  0.5468, -0.0939]) tensor(5.7109e-05)
tensor(3.1997) tensor(3.1359)
[Test Split] Check score before calibration=3.199695587158203, Check score after quantile calibration=3.1359283924102783, Check score after dist calibration=2.9658091068267822
[Test Split] Calibration score before calibration=0.03905351046967815, Calibration score after quantile calibration=0.0002876844

In [18]:
quantile_calibrator.inverse_recalibrator.predict(torch.Tensor(np.linspace(0, 1, num=num_buckets)))

array([4.30345535e-05, 1.05764703e-01, 1.57659359e-01, 1.97080797e-01,
       2.31788295e-01, 2.62724499e-01, 2.94475439e-01, 3.24760172e-01,
       3.58738512e-01, 4.00818915e-01, 4.39831724e-01, 4.83572265e-01,
       5.29090189e-01, 5.84218020e-01, 6.46472821e-01, 7.16700115e-01,
       7.94666101e-01, 8.84678976e-01, 9.67842391e-01, 1.00000000e+00])

In [13]:
torch.Tensor(np.linspace(0, 1, num=num_buckets))
# num_buckets

tensor([0.0000, 0.0526, 0.1053, 0.1579, 0.2105, 0.2632, 0.3158, 0.3684, 0.4211,
        0.4737, 0.5263, 0.5789, 0.6316, 0.6842, 0.7368, 0.7895, 0.8421, 0.8947,
        0.9474, 1.0000])

In [19]:
print(input_cdf.shape, empirical_cdf.shape)
print(input_cdf)
print(empirical_cdf)

torch.Size([4128]) torch.Size([4128])
tensor([0.1040, 0.2976, 0.5547,  ..., 0.7264, 0.3942, 0.2152])
tensor([0.0516, 0.3203, 0.6555,  ..., 0.7972, 0.4625, 0.1865])
